In [101]:
import re
import requests
from bs4 import BeautifulSoup as bs
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import simplejson as json

In [ ]:
res = requests.get("https://piaofang.maoyan.com/dashboard/movie")
res.encoding = 'utf-8'
soup = bs(res.text, 'html.parser')

In [ ]:
browserOptions = Options()
#browserOptions.add_argument("--headless")

capa = DesiredCapabilities.CHROME
capa["pageLoadStrategy"] = "none"
capa["goog:loggingPrefs"] = {"performance": "ALL"}
driver = webdriver.Chrome(desired_capabilities=capa)
wait = WebDriverWait(driver, 20)

In [ ]:
#create snapshot of the entire page to prevent it from constantly changing
driver.get("https://piaofang.maoyan.com/dashboard/movie")
test = None
while not test:
    try:
        test = wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'moviename-td')))
    except:
        driver.refresh();
        
logs_raw = driver.get_log("performance")
logs = [json.loads(lr["message"])["message"] for lr in logs_raw]

def log_filter(log_):
    return (
        # is an actual response
        log_["method"] == "Network.responseReceived"
        # and json
        and "json" in log_["params"]["response"]["mimeType"]
    )

responses = []

for log in filter(log_filter, logs):
    request_id = log["params"]["requestId"]
    resp_url = log["params"]["response"]["url"]
    print(f"Caught {resp_url}")
    response = driver.execute_cdp_cmd("Network.getResponseBody", {"requestId": request_id})
    responses.append(response)
    

    

In [ ]:
body0 = json.loads(responses[0]['body'])
movieList = body0['movieList']['list']
date = body0['calendar']['today']
font_url = body0['fontStyle'].split('"')[-2]
font_url

In [ ]:
# Get reference fonts from the file tree
from fontTools.ttLib import TTFont
headers = {
            "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_4) AppleWebKit/537.36 (KHTML, like Gecko) "
              "Chrome/66.0.3359.139 Safari/537.36 "
    }

woff_url = 'http:' + font_url
response_woff = requests.get(woff_url, headers=headers).content

print("Woff retrieval succuessful: " + str(len(response_woff) > 0))

with open('fonts.woff', 'wb') as f:
    f.write(response_woff)

# Already saved, DO NOT RUN THE FOLLOWING 
#with open('basefonts.woff', 'wb') as f:
    #f.write(response_woff)
    
#font1 = TTFont('basefonts.woff')
#font1.saveXML('basefonts.xml')

In [ ]:
import pandas as pd
df = pd.DataFrame.from_records(movieList)

In [ ]:
df['boxSplitUnit'][4]

In [139]:
# Now starts the tedius process of mapping the new fonts to the contours and then to the numbers
from xml.etree import ElementTree
from difflib import get_close_matches
from difflib import SequenceMatcher
from fontTools.misc.xmlWriter import XMLWriter
import re

# Create Bijection between numbers and unicode 
unicodeToInt = {"uniF581": 0, "uniF56B": 1, "uniE46D": 2, "uniE13D": 3, "uniEAA8": 4, 
                "uniE808": 5, "uniE1D4": 6, "uniE66A": 7, "uniE43F": 8, "uniECDC": 9}

def hexToUnicode(hexa):
    return 'uni' + hexa[3:].upper()

def uniToHex(uni):
    return "&#x" + uni[3:].lower()

def base10toHex(num):
    return "&#x" + str(int(num, 16))

def elemToStr(elem):
    return ElementTree.tostring(elem, encoding='utf8', method='xml')

def trimXML(xml):
    return ''.join(xml.split())

#If the regular equality doesn't work we then have to use fuzzy matching
xRg = re.compile(r'x=\"([0-9]+)\"')
yRg = re.compile(r'y=\"([0-9]+)\"')
def getCoFromXML(xml):
    stringToMatch = trimXML(xml)
    print(stringToMatch)
    x = xRg.match(stringToMatch)
    y = yRg.match(stringToMatch)
    return {"x": x, "y": y}
    
def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()
    

## Getting digits from the font data
with neural networks

In [166]:
from fontTools.ttLib import TTFont
from PIL import ImageFont, Image, ImageDraw
import pytesseract
import cv2
import numpy as np
import digit_recognition as dr
from wand.image import Image as wImage
import random

In [173]:
# Creating training sets and labels

def distortImage(number):
    with wImage(filename=("training_images/" + str(i) + ".png")) as img:
        scale = random.uniform(0.6, 1.4)
        img.distort('scale_rotate_translate', (scale, 0,))
        img.save(filename='test.png')
        return img

# Reference Images
basefontimgs = []
basefontlabels = []
for i in range(10):
    with Image.open("training_images/" + str(i) + ".png") as im:
        basefontimgs.append(np.array(im))
        scale = 0.5
        
    basefontlabels.append([0.01 if x != i else 0.09 for x in range(10)])



distortImage(4)


ClosedImageError: <wand.image.Image: (closed)> is closed already

<wand.image.Image: (closed)>

In [146]:
# Import data
train_images, train_labels, test_images, test_labels = dr.pre_processing()

# Build neural network (with two hidden layers of 200/100 nodes respectively)
neural_network = dr.NeuralNetwork([784,200,100,10], bias=True)
neural_network.train(train_images, train_labels, epochs=10)
neural_network.evaluate(test_images, test_labels)

Data has been downloaded
Data preprocessed successfully
Network created successfully
Training...
Epoch: 1 (of 10)
Epoch: 2 (of 10)
Epoch: 3 (of 10)
Epoch: 4 (of 10)
Epoch: 5 (of 10)
Epoch: 6 (of 10)
Epoch: 7 (of 10)
Epoch: 8 (of 10)
Epoch: 9 (of 10)
Epoch: 10 (of 10)
Network trained successfully
Accuracy: 97.59%
Recall for 0: 99.08%
Precision for 0: 97.29%
Recall for 1: 99.38%
Precision for 1: 98.69%
Recall for 2: 97.38%
Precision for 2: 97.38%
Recall for 3: 97.72%
Precision for 3: 98.01%
Recall for 4: 97.76%
Precision for 4: 96.68%
Recall for 5: 96.97%
Precision for 5: 97.74%
Recall for 6: 98.02%
Precision for 6: 97.71%
Recall for 7: 97.37%
Precision for 7: 97.37%
Recall for 8: 96.00%
Precision for 8: 97.40%
Recall for 9: 95.94%
Precision for 9: 97.48%


In [150]:
# train specifically for our basefonts
basefontimgs = []
basefontlabels = []
for i in range(10):
    with Image.open("training_images/" + str(i) + ".png") as im:
        basefontimgs.append(np.array(im))
    basefontlabels.append([0.01 if x != i else 0.09 for x in range(10)])
neural_network.train(basefontimgs, basefontlabels, epochs = 100)

Training...
Epoch: 1 (of 100)
Epoch: 2 (of 100)
Epoch: 3 (of 100)
Epoch: 4 (of 100)
Epoch: 5 (of 100)
Epoch: 6 (of 100)
Epoch: 7 (of 100)
Epoch: 8 (of 100)
Epoch: 9 (of 100)
Epoch: 10 (of 100)
Epoch: 11 (of 100)
Epoch: 12 (of 100)
Epoch: 13 (of 100)
Epoch: 14 (of 100)
Epoch: 15 (of 100)
Epoch: 16 (of 100)
Epoch: 17 (of 100)
Epoch: 18 (of 100)
Epoch: 19 (of 100)
Epoch: 20 (of 100)
Epoch: 21 (of 100)
Epoch: 22 (of 100)
Epoch: 23 (of 100)
Epoch: 24 (of 100)
Epoch: 25 (of 100)
Epoch: 26 (of 100)
Epoch: 27 (of 100)
Epoch: 28 (of 100)
Epoch: 29 (of 100)
Epoch: 30 (of 100)
Epoch: 31 (of 100)
Epoch: 32 (of 100)
Epoch: 33 (of 100)
Epoch: 34 (of 100)
Epoch: 35 (of 100)
Epoch: 36 (of 100)
Epoch: 37 (of 100)
Epoch: 38 (of 100)
Epoch: 39 (of 100)
Epoch: 40 (of 100)
Epoch: 41 (of 100)
Epoch: 42 (of 100)
Epoch: 43 (of 100)
Epoch: 44 (of 100)
Epoch: 45 (of 100)
Epoch: 46 (of 100)
Epoch: 47 (of 100)
Epoch: 48 (of 100)
Epoch: 49 (of 100)
Epoch: 50 (of 100)
Epoch: 51 (of 100)
Epoch: 52 (of 100)
Epoch: 53

In [130]:
basefontlabels

[[0.09, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01],
 [0.01, 0.09, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01],
 [0.01, 0.01, 0.09, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01],
 [0.01, 0.01, 0.01, 0.09, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01],
 [0.01, 0.01, 0.01, 0.01, 0.09, 0.01, 0.01, 0.01, 0.01, 0.01],
 [0.01, 0.01, 0.01, 0.01, 0.01, 0.09, 0.01, 0.01, 0.01, 0.01],
 [0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.09, 0.01, 0.01, 0.01],
 [0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.09, 0.01, 0.01],
 [0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.09, 0.01],
 [0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.09]]

In [151]:
def uni_2_png_stream(txt: str, font: str, img_size=28):
    img = Image.new('1', (img_size, img_size), 255) 
    draw = ImageDraw.Draw(img)
    font = ImageFont.truetype(font, int(img_size * 0.7)) 

    txt = chr(txt)
    x, y = draw.textsize(txt, font=font) 
    draw.text(((img_size - x) // 2, (img_size - y) // 2), txt, font=font, fill=0)
    img.save(txt + '.png')
    return img 

def predict(unicode, fontFile):
    
    image = uni_2_png_stream(int(unicode[3:], 16), fontFile)
    matrix_form = np.array(image)
    weighted_predictions = np.ndarray.flatten(neural_network.run(matrix_form))
    most_possible = np.argmax(weighted_predictions)
    return most_possible


filename = 'basefonts.woff'  # 字体文件的路径信息
f = TTFont(filename)
for x in f.getGlyphNames()[1:-1]:
    print(x)
    print(predict(x, filename))
    


2 extra bytes in post.stringData array


uniE13D
3
uniE1D4
6
uniE43F
8
uniE46D
2
uniE66A
7
uniE808
5
uniEAA8
4
uniECDC
9
uniF56B
6
uniF581
0


In [152]:
# Map contours to numbers
baseFonts = TTFont('basefonts.woff')
currentFonts = TTFont('fonts.woff')
hexToInt = {}

for j in currentFonts.getGlyphNames()[1:-1]:
    hexToInt[uniToHex(j)] = predict(j, 'fonts.woff')

hexToInt

2 extra bytes in post.stringData array


{'&#xe082': 6,
 '&#xe455': 5,
 '&#xe51b': 3,
 '&#xe945': 7,
 '&#xed1c': 3,
 '&#xedef': 5,
 '&#xf23b': 4,
 '&#xf365': 5,
 '&#xf7a0': 3,
 '&#xf7d8': 3}

In [ ]:
unitLookup = {'百': 100, '千': 1000, '万': 10000, '亿': 1*10**8}

#converts the weird character to a float
def convertToFloat(string):
    spCharLst = string.split(';')
    result = ''
    for i in spCharLst:
        if len(i) > 7: #has a dot in front
            result += '.' + str(hexToInt[i[1:]])
        elif len(i) == 7: #in case of bad parsing
            result += str(hexToInt[i])
    return float(result)

#helper function for converting the entire block to a single int
def convertDictToInt(dictionary):
    return int(convertToFloat(dictionary['num']) * unitLookup[dictionary['unit']])


In [ ]:
df['boxSplitUnit'] = df['boxSplitUnit'].apply(convertDictToInt)
df['splitBoxSplitUnit'] = df['splitBoxSplitUnit'].apply(convertDictToInt)
df['movieInfo'] = df['movieInfo'].apply(lambda x : x['movieName'])
df

In [ ]:
#大盘
body0['movieList']['nationBoxInfo']